# TensorBoard可视化

## 1. 安装

In [ ]:
!pip install tensorboardX

## 2. TensorBoard可视化的基本逻辑
- 将TensorBoard看做一个记录员，它可以记录我们指定的数据，包括模型每一层的feature map，权重，以及训练loss等等。
- TensorBoard将记录下来的内容保存在一个用户指定的文件夹里，程序不断运行中TensorBoard会不断记录。记录下的内容可以通过网页的形式加以可视化。

## 3. TensorBoard的配置与启动

指定一个文件夹保存记录下来的数据

In [2]:
# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter('./runs')

启动Tensorboard

In [ ]:
!tensorboard --logdir=/path/to/logs/ --port=xxxx

## 4. TensorBoard可视化

模型结构可视化

In [3]:
import torch.nn as nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.adaptive_pool = nn.AdaptiveMaxPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        y = self.sigmoid(x)
        return y


model = Net()
print(model)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (adaptive_pool): AdaptiveMaxPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=64, out_features=32, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [5]:
import torch
writer.add_graph(model, input_to_model=torch.rand(1, 3, 224, 224))
writer.close()

In [ ]:
!tensorboard --logdir=pytorch_datawhale/06_pytorch_visual/runs --port=6006

图像可视化

In [ ]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform_train = transforms.Compose(
    [transforms.ToTensor()])
transform_test = transforms.Compose(
    [transforms.ToTensor()])

train_data = datasets.CIFAR10(
    ".", train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR10(
    ".", train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

images, labels = next(iter(train_loader))

# 仅查看一张图片
writer = SummaryWriter('./pytorch_tb')
writer.add_image('images[0]', images[0])
writer.close()

# 将多张图片拼接成一张图片，中间用黑色网格分割
# create grid of images
writer = SummaryWriter('./pytorch_tb')
img_grid = torchvision.utils.make_grid(images)
writer.add_image('image_grid', img_grid)
writer.close()

# 将多张图片直接写入
writer = SummaryWriter('./pytorch_tb')
writer.add_images("images", images, global_step=0)
writer.close()

连续变量可视化

In [ ]:
writer = SummaryWriter('./pytorch_tb')
for i in range(500):
    x = i
    y = x**2
    writer.add_scalar("x", x, i)  # 日志中记录x在第step i 的值
    writer.add_scalar("y", y, i)  # 日志中记录y在第step i 的值
writer.close()

In [ ]:
# 同一图中多条曲线
writer1 = SummaryWriter('./pytorch_tb/x')
writer2 = SummaryWriter('./pytorch_tb/y')
for i in range(500):
    x = i
    y = x*2
    writer1.add_scalar("same", x, i)  # 日志中记录x在第step i 的值
    writer2.add_scalar("same", y, i)  # 日志中记录y在第step i 的值
writer1.close()
writer2.close()

参数分布可视化

In [ ]:
import torch
import numpy as np

# 创建正态分布的张量模拟参数矩阵


def norm(mean, std):
    t = std * torch.randn((100, 20)) + mean
    return t


writer = SummaryWriter('./pytorch_tb/')
for step, mean in enumerate(range(-10, 10, 1)):
    w = norm(mean, 1)
    writer.add_histogram("w", w, step)
    writer.flush()
writer.close()

## 5. 服务器端使用

### ssh

1. 将服务器的6006端口重定向到自己机器上来，在本地的终端里输入以下代码：其中16006代表映射到本地的端口，6006代表的是服务器上的端口。
```
ssh -L 16006:127.0.0.1:6006 username@remote_server_ip
```
2. 在服务上使用默认的6006端口正常启动tensorboard
```
tensorboard --logdir=your_log_dir --port=6006
```
3. 在本地的浏览器输入地址
```
127.0.0.1:16006 或者 localhost:16006
```